In [ ]:
from torch.ao.nn.quantized.functional import threshold

from models.microscopy_cnn import MicroscopyCNN
from scripts.model_metrics import score_model
import torch
from scripts.dataset_loader import MicroscopyDataset
from torch.utils.data import DataLoader

In [ ]:
model = MicroscopyCNN(in_channels=2, task='classification')

# model.load_state_dict(torch.load(r'C:\Users\nmp002\PycharmProjects\MPM-MachineLearning\trained_models\best_NADH_only_24FEB2025.pt', map_location=torch.device('cpu'),weights_only=True))
model.load_state_dict(torch.load(r'C:\Users\nmp002\PycharmProjects\MPM-MachineLearning\trained_models\epoch250_NADH_SHG.pt', map_location=torch.device('cpu'),weights_only=True))

In [ ]:
# Load data
csv_file=r"C:\Users\nmp002\PycharmProjects\MPM-MachineLearning\data\newData\labels.csv"
root_dir=r"C:\Users\nmp002\PycharmProjects\MPM-MachineLearning\data\newData"

dataset = MicroscopyDataset(
    csv_file=csv_file,
    root_dir=root_dir,
    # channels = ['nadh'],
    channels = ['nadh', 'shg'],
    transform=None
)

In [ ]:
specific_test_paths = []
# sample_indices = [23,0,3,20]
sample_indices = range(0,29)
eval_indices = [27, 4, 25, 7, 8]
for sample_index in eval_indices:
    if sample_index not in [25,7]:
        specific_test_paths.append(dataset.sample_wise_paths[sample_index])

print(specific_test_paths)
# print('\n------------------------------------------\n')
# print(dataset.sample_wise_paths)

In [ ]:
from sklearn.metrics import roc_auc_score


def score_em(t, o):
    fpr, tpr, thresholds = roc_curve(t, o)
    score = auc(fpr, tpr)
    score_all = roc_auc_score(t, o)
    print(score_all)
    thresh = thresholds[np.argmax(tpr - fpr)]
    print(type(thresholds), type(thresh))
    preds = [out >= thresh for out in o]
    for tar,pred in zip(t, preds):
        print(f'{tar} -> {pred}')
    RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=score).plot()
    ConfusionMatrixDisplay.from_predictions(t, preds).plot()

In [ ]:
from sklearn.metrics import auc, roc_curve, confusion_matrix, RocCurveDisplay, ConfusionMatrixDisplay
from scripts.dataset_loader import tiff_to_tensor
import numpy as np
with torch.no_grad():
    model.eval()
    targets = []
    all_targets = []
    outs = []
    all_outs = []
    for sample in specific_test_paths:
        fov_outs = []
        if sample:
            for fov in sample:
                # De-nest fov paths and get the indexed item path
                score = fov[1]
                combined_image = torch.cat([tiff_to_tensor(channel) for channel in fov[0]], dim=0).unsqueeze(0)
                y = model(combined_image).item()
                all_outs.append(y)
                fov_outs.append(y)
                all_targets.append(1 if score > 25 else 0)
            targets.append(1 if score > 25 else 0)
            outs.append(np.mean(fov_outs))
score_em(all_targets, all_outs)
score_em(targets, outs)

In [ ]:
print(all_outs)

In [ ]:
# best NADH_SHG = 0.5153061224489796
# epoch500 NADH_SHG =

In [ ]:
print(fov_outs)

In [ ]:
print(targets)

In [ ]:
print(outs)